## Eigenfaces

This problem is a bit more challenging and as such, will not count as much on the HW (points wise). For more information on this problem, see my course notes.

While we are extremely good at distinguishing faces (most of us), different faces share much in common. We obviously have two eyes, a nose, etc.. But how can we get a compute to find a set of features that describe faces? And how can we use these to dimensionaly compress faces? One of the early techniques to do this utilized the SVD.

In the following, I have provided code to import ~2k faces from the Yale faces database. You will need to import some packages below. This imports the data into a 2339x192x192 matrix. The first index is the number of faces and the second indices are the size of the image in pixels (192x192). Since this type of 3D matrix is not of use to us, we will need to unwrap each 2D image into a 1D image of pixels. I have provided some utility functions below to unwrap the images into vectors and to reformat them into matrices. The vectors are useful for analysis while we need them in 2D matrix form for plotting the faces themselves. All linear algebra will happen on the 2339x36864 matrix (2D) of vector faces while all plotting will happen on the 2339x192x192 matrix (3D).

Your job is the following. We will be using the SVD to break this database of faces into features, discard less important features, and reconstruct the faces from these lower dimensional representations.

1) Just plot a face or two to get a sense of what these images look like.
2) Compute the "mean face". If each face is a 2D matrix, just calculate the mean of all the 2D matrices. Plot this just to get a sense of what it looks like.
3) Subtract that mean face from each individual face. This is mean centering. Plot a mean centered face just to get a sense of what it looks like.
4) Take this mean centered collection of faces and transform them from a 3D to a 2D matrix as discussed above. This is what we will need for the SVD.
5) Compute the SVD.
6) Plot the singular values. We are trying to see how quickly they decrease in magnitude.
7) Plot a few eigenfaces. Remember, the eigenfaces are the column vectors of V. To plot the n-th eigenface, extract the n-th column of V, reshape it into a 2D matrix, then plot that.
8) Now lets compress the data. We are only going to keep the first 500 dimensions of the SVD. We worked with this in one of the other HW problems so I won't go into as much detail here. Remove all but the first 500 columns of U and V, and all but the first 500 singular values of D. Recompute the SVD to get a new 2339x36864 matrix (2D) of faces. Note, that before computing the SVD, we subtracted the "mean face" from each face. The recomputed compressed SVD thus compresses these faces in the "mean centered" sense. To get true faces back, we need to take each recomputed face in this recomputed SVD, and add the mean face back to it. Lets call the original matrix of faces X and the new one Xstar (after adding the mean face back).
9) Lets look at the first face. Plot the first face contained in X and the first face contained in Xstar. You may have to reshape these back into a 2D (192x192) matrix for plotting. They should look very similar.


In [ ]:
using Pkg
Pkg.status()
#Pkg.add("Images")
#Pkg.add("ImageFeatures")
#Pkg.add("BenchmarkTools")


using Images, ImageFeatures 
using LinearAlgebra
using BenchmarkTools
using Plots

In [ ]:
# Pull in the data. This will produce a 2339x192x192 matrix where there are 
# 2339 images each of size 192x192.
dim = 192

raw_imgs = zeros(2339, dim, dim);
path_train = "./Faces_datasets/";
count = 1
for directory = readdir(path_train)
    if directory == ".DS_Store" # My computer puts this in my directory and we need to skip it.
        continue
    end
    
    for file = readdir(path_train*directory)
        # println(path_train*directory*"/"*file)
        raw_imgs[count,:,:] =  load(path_train*directory*"/"*file)
        count += 1
    end
end

In [ ]:
# Show one face
p1 = Gray.(raw_imgs[1,:,:]) 

In [ ]:
mean_face = mean(raw_imgs, dims=1)
mean_face = reshape(mean_face, 192, 192)
p2 = Gray.(mean_face)


# Subtract the mean face from each of the individual faces
for i = 1:size(raw_imgs)[1]
    raw_imgs[i,:,:] = raw_imgs[i,:,:] .- mean_face
end

#Take this mean centered collection of faces and transform them from a 3D to a 2D matrix as discussed above. This is what we will need for the SVD.
X = reshape(raw_imgs, size(raw_imgs)[1], size(raw_imgs)[2]*size(raw_imgs)[3])

In [ ]:
# After analysis, we will need to convert the image vectors back to image matrices for plotting.
# This takes the previous transform in reverse.
function reshape_face_2D(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], trunc(Int64, sqrt(dims[2])), trunc(Int64, sqrt(dims[2]))))
end



# Perform the SVD on the mean centered faces
U, S, V = svd(X)

# Plot the first 10 eigenfaces
eigenfaces = reshape_face_2D(V[1:10,:])
size(eigenfaces)
#p3 = plot(Gray.(eigenfaces[1,:,:]), layout=(2,5), title="First 10 Eigenfaces", size=(800,400))



In [ ]:


# Example plot of a mean-centered face
p_mean_centered = Gray.(mean_centered_imgs[1, :, :])
plot(p_mean_centered)

# 4) Transform from 3D to 2D matrix for SVD

function reshape_face_1D(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], dims[2]*dims[3]))
end

X = reshape_face_1D(mean_centered_imgs)

# 5) Compute SVD
U, S, Vt = svd(X)

# 6) Plot the singular values
plot(S, yscale=:log10, title="Singular Values", xlabel="Index", ylabel="Value")

# 7) Plot a few eigenfaces
function plot_eigenface(V, n)
    eigenface = reshape_face_2D(V[:, n])
    p_eigen = Gray.(eigenface)
    plot(p_eigen)
end

# Plot the first 3 eigenfaces
plot_eigenface(Vt', 1)
plot_eigenface(Vt', 2)
plot_eigenface(Vt', 3)

#Compress the data and keep only the first 500 dimensions
Ustar = U[:, 1:500]
Sstar = S[1:500]
Vstar = Vt[1:500, :]'

# Recompute the SVD to get a new matrix of faces
Xstar = Ustar * Diagonal(Sstar) * Vstar

end

# Call the function to compare the first face from the original and reconstructed data
compare_faces(X, Xstar_reconstructed, 1)

# Ensure mean_face is a vector for the addition
mean_face_vector = reshape(mean_face_2d, 192*192)

# Add the mean face back to each reconstructed face
Xstar_reconstructed = eachrow(Xstar) .+ mean_face_vector


function reshape_face_2D(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], trunc(Int64, sqrt(dims[2])), trunc(Int64, sqrt(dims[2]))))
end
# Function to compare faces might need adjustment based on the specific error context
# Here's an example assuming Xstar_reconstructed and the original data need reshaping
function compare_faces(original_idx, reconstructed_idx)
    original_face = reshape_face_2D(original[original_idx, :])
    reconstructed_face = reshape_face_2D(reconstructed[reconstructed_idx, :])
    plot(layout=(1, 2), size=(600, 300))
    plot!(subplot=1, Gray.(original_face))
    plot!(subplot=2, Gray.(reconstructed_face))
end

# Comparing the first original and reconstructed face
compare_faces(1, 1)


In [ ]:
# Utility functions. These functions convert the entire collection of faces back and forth at once.
# They do not convert them one face at a time.

# When we store a bank of images, each one of them needs to be converted from a 192x192 square 
# to a 1x36864 row vector. So we go from having a 3-Dim matrix to a 2-Dim matrix
function reshape_face_1D(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], dims[2]*dims[3]))
end

# After analysis, we will need to convert the image vectors back to image matrices for plotting.
# This takes the previous transform in reverse.
function reshape_face_2D(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], trunc(Int64, sqrt(dims[2])), trunc(Int64, sqrt(dims[2]))))
end

In [ ]:
# Reshape the 3D faces matrix into a 2D faces matrix

# Calculate the SVD. This takes 20-60 seconds.

# V containes the "eigenfaces". We need to convert these from a 2D --> 3D matrix for plotting.
eigenfaces = reshape_face_2D(V'); # The eigenfaces



In [ ]:
# Plot a few eigenfaces. Note that the ones associated with larger singular values have structure
# while the ones with lower singular values have less structure. # This is expected since low 
# singular values are more associated with noise.

# Plot 2, 3, 100, 2100
# Note that it is useful to scale all the eigenfaces to the [0,1] interval for visual purposes.
Gray.(eigenfaces[plot_ind,:,:] ./ maximum(eigenfaces[plot_ind,:,:])) 

In [ ]:
# Plot singular values and show that they go way down.
#plot( INSERT SINGULAR VALUES HERE ,yscale = :log10 , ylimits = (10^-4,10^4));
xlabel!("Singular Value #");
ylabel!("Singular Value magnitude")

In [ ]:
# Let's perform compression. We're going to keep the 500 largest singular values and eigenfaces.
# Truncate U, Σ, and V to 500 dimensions. Multiply these together. The resulting matrix is a compressed
# version of your raw faces. We need to reshape these reduced faces to 2D (for each face) and add the 
# previously subtracted mean face back on.

# Note that the svd function produces the singular values as a vector. You need to truncate this vector
# to 500 values, then turn it into a diagonal matrix. You can use the Diagonal() function for this.

# n_eigen = 500
# ...
# ...
# reduced_faces = reshape_face_2D(X_red);
# Gray.(reduced_faces[1,:,:] .+ mean_face) # Plot the first reduced face


In [ ]:
Gray.(raw_imgs[1,:,:]) # Plot the raw image for the first face